In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import tweepy
from pandas.io.json import json_normalize
import sys
import jsonpickle
import os

In [0]:
atoken = "1211674579879391233-VwxwWZO0nnio8wwBVzncqhjfCTmz8S"
asecret = "SUNhejwVlBrsx1B8I7yqXkn9NzjOKR6kzejkAHmJHdOTP"
ckey = "nSLLEuipKq6iJetBCLj33mv1J"
csecret = "s9XGj7t3XMvsmX2EHWeUHex2VvthapZFmmpFAj40VsQV2SvnVg"

In [0]:
# Replace the API_KEY and API_SECRET with your application's key and secret.
auth = tweepy.AppAuthHandler(ckey, csecret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [0]:
companies = dataiku.get_custom_variables(typed=True)["company"]

In [72]:
df = pd.DataFrame()

for company in companies:
    print(company)
    searchQuery = company + " -RT"# this is what we're searching for
    print("search query :", searchQuery)
    maxTweets = 100 # Some arbitrary large number
    tweetsPerQry = 100  # this is the max the API permits
    tweepy_REST_API = dataiku.Folder("tweepy_REST_API")
    folder_path = tweepy_REST_API.get_path()
    fName = folder_path + '/tweets.txt' # We'll store the tweets in a text file.


    # If results from a specific ID onwards are reqd, set since_id to that ID.
    # else default to no lower limit, go as far back as API allows
    sinceId = None

    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1

    tweetCount = 0


    # -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
    print("Downloading max {0} tweets".format(maxTweets))
    with open(fName, 'w') as f:
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
                    else:
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                                since_id=sinceId, lang="en")
                else:
                    if (not sinceId):
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                                max_id=str(max_id - 1), lang="en")
                    else:
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId, lang="en")
                if not new_tweets:
                    print("No more tweets found")
                    break
                for tweet in new_tweets:
                    f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
                    
                    if len(tweet._json['entities']['urls']) == 0:
                        links = ""
                    else: 
                        links = tweet._json['entities']['urls'][0]["expanded_url"]
                        
                    if tweet._json["place"]==None:
                        country = ""
                    else:
                        country = tweet._json["place"]['country']
                        
                    if tweet._json["geo"] != None:
                        coordinates = str(tweet._json["geo"]['coordinates'])
                    else:
                        coordinates = ""

                    new_row = pd.DataFrame({'timestamp':tweet._json['created_at'],
                                            'tweet_id':[tweet._json["id"]],
                                            'text':tweet._json["text"],
                                            'hashtags':[tweet._json["entities"]["hashtags"]],
                                            "links":links,
                                            # 'user_mentions':tweet._json["entities"]["user_mentions"][0]["screen_name"],
                                            # 'user_mentions_id':tweet._json["entities"]["user_mentions"][0]["id"],
                                            # 'user_mentions_indices':[tweet._json["entities"]["user_mentions"][0]["indices"]],
                                            'in_reply_to_status_id':tweet._json["in_reply_to_status_id"],
                                            'in_reply_to_user_id':tweet._json["in_reply_to_user_id"],
                                            'in_reply_to_screen_name':tweet._json["in_reply_to_screen_name"],
                                            'user_id':tweet._json["user"]["id"],
                                            'user_name':tweet._json["user"]["name"],
                                            'user_screen_name':tweet._json["user"]["screen_name"],
                                            'user_location':tweet._json["user"]["location"],
                                            'followers_count':tweet._json["user"]["followers_count"],
                                            'friends_count':tweet._json["user"]["friends_count"],
                                            'user_creation':tweet._json["user"]["created_at"],
                                            'favourites_count':tweet._json["user"]["favourites_count"],
                                            'coordinates':coordinates,
                                            # 'geo':tweet._json["geo"],
                                            'country':country,
                                            'retweet_count':tweet._json["retweet_count"],
                                            'retweeted':tweet._json["retweeted"],
                                            'lang':tweet._json["lang"]
                                        })
                    """new_row = json_normalize(tweet._json)[[ # 'contributors',
                                                           # "coordinates",
                                                           "created_at",
                                                           'entities.hashtags', 'entities.urls',
                                                           # 'geo',
                                                           'id',
                                                           "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                                           "lang",
                                                           # 'place.country', 'place.country_code', 'place.full_name', 'place.id','place.name', 'place.place_type', 'place.url',
                                                           'retweet_count', 'retweeted','text','user.created_at',
                                                           'user.followers_count', 'user.following','user.id', 'user.screen_name',
                                                           'user.time_zone']]"""
                    new_row["company"] = company
                    df = df.append(new_row, ignore_index = True)

                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id
            except tweepy.TweepError as e:
                # Just exit if any error
                print("some error : " + str(e))
                break

    print ("Downloaded {0} tweets for {1}, Saved to {2}".format(tweetCount, company, fName))

Air France
search query : Air France -RT
Downloaded 84 tweets
Downloaded 180 tweets
Downloaded 180 tweets for Air France, Saved to /Users/mmiyazaki/dataiku/Design/DATA_DIR/managed_folders/ANALYSISOFCUSTOMERFRUSTRATIONSINAIRLINEINDUSTRIES/2o8aWzsk/tweets.txt
Lufthansa
search query : Lufthansa -RT
Downloaded 100 tweets
Downloaded 100 tweets for Lufthansa, Saved to /Users/mmiyazaki/dataiku/Design/DATA_DIR/managed_folders/ANALYSISOFCUSTOMERFRUSTRATIONSINAIRLINEINDUSTRIES/2o8aWzsk/tweets.txt
Ryanair
search query : Ryanair -RT
Downloaded 100 tweets
Downloaded 100 tweets for Ryanair, Saved to /Users/mmiyazaki/dataiku/Design/DATA_DIR/managed_folders/ANALYSISOFCUSTOMERFRUSTRATIONSINAIRLINEINDUSTRIES/2o8aWzsk/tweets.txt
easyJet
search query : easyJet -RT
Downloaded 100 tweets
Downloaded 100 tweets for easyJet, Saved to /Users/mmiyazaki/dataiku/Design/DATA_DIR/managed_folders/ANALYSISOFCUSTOMERFRUSTRATIONSINAIRLINEINDUSTRIES/2o8aWzsk/tweets.txt
United Airlines
search query : United Airlines -RT
D

In [73]:
df.head()

,timestamp,tweet_id,text,hashtags,links,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,user_name,user_screen_name,user_location,followers_count,friends_count,user_creation,favourites_count,coordinates,country,retweet_count,retweeted,lang,company
0,Mon May 04 17:36:45 +0000 2020,1257363549476847618,"@zabulka @SimonCalder Soooo agree. S Korea, Cz...",[],https://twitter.com/i/web/status/1257363549476...,1256964215727890432,55310845,zabulka,2227929828,Frances Gray,Franki2013Gray,"Tadworth, Surrey",109,284,Tue Dec 03 08:06:45 +0000 2013,1882,,,0,False,en,Air France
1,Mon May 04 17:33:33 +0000 2020,1257362742488567817,@Bardia84261966 The one one the right is at la...,[],https://twitter.com/i/web/status/1257362742488...,1257352075089973249,1237435796820557824,Bardia84261966,1039530318921908225,Lazy 'Counterpart',SuperLazy6,,59,258,Tue Sep 11 15:05:16 +0000 2018,2545,,,0,False,en,Air France
2,Mon May 04 17:33:03 +0000 2020,1257362620274933769,Ryanair boss will not be happy with Air France...,[],https://youtu.be/Dh-2McZe2KU,None,None,None,1106193729235402753,🇬🇧 𝗕𝗕 🇬🇧,BrexitBetrayed,United Kingdom 🇬🇧,20912,6256,Thu Mar 14 14:01:51 +0000 2019,24468,,,1,False,en,Air France
3,Mon May 04 17:31:32 +0000 2020,1257362235464318980,Coronavirus aid: Air France 'must cut domestic...,[],http://dlvr.it/RW0HNY,None,None,None,60180662,Emad Rassmy,erassmy,"Palm Desert, California",357,171,Sat Jul 25 23:49:57 +0000 2009,12,,,0,False,en,Air France
4,Mon May 04 17:31:10 +0000 2020,1257362143919431681,Ryanair boss will not be happy with Air France...,[],https://youtu.be/Dh-2McZe2KU,None,None,None,857696952884097025,Hugh de Morville,MorvilleHugh,,203,175,Thu Apr 27 20:44:23 +0000 2017,12981,,,0,False,en,Air France


In [28]:
py_recipe_output = dataiku.Dataset("tweepy_REST_API")
py_recipe_output.write_with_schema(df)

582 rows successfully written (WqsFZ0aYQs)
